In [167]:
import cv2
import numpy as np
import scipy.io
from scipy import ndimage
import os
from PIL import Image

In [168]:
# 读取 mat 文件，打印一下变量名称

mat_data = scipy.io.loadmat('data/FADE_key_variables.mat')

for key in mat_data.keys():
    if not key.startswith('__'):  # 过滤掉MATLAB的元数据
        print(f"- {key}: {type(mat_data[key])}, shape: {mat_data[key].shape if hasattr(mat_data[key], 'shape') else 'N/A'}")

- I: <class 'numpy.ndarray'>, shape: (768, 576, 3)
- D: <class 'numpy.ndarray'>, shape: (1, 1)
- D_map: <class 'numpy.ndarray'>, shape: (96, 72)
- ps: <class 'numpy.ndarray'>, shape: (1, 1)
- row: <class 'numpy.ndarray'>, shape: (1, 1)
- col: <class 'numpy.ndarray'>, shape: (1, 1)
- dim: <class 'numpy.ndarray'>, shape: (1, 1)
- patch_row_num: <class 'numpy.ndarray'>, shape: (1, 1)
- patch_col_num: <class 'numpy.ndarray'>, shape: (1, 1)
- R: <class 'numpy.ndarray'>, shape: (768, 576)
- G: <class 'numpy.ndarray'>, shape: (768, 576)
- B: <class 'numpy.ndarray'>, shape: (768, 576)
- Ig: <class 'numpy.ndarray'>, shape: (768, 576)
- Irn: <class 'numpy.ndarray'>, shape: (768, 576)
- Ign: <class 'numpy.ndarray'>, shape: (768, 576)
- Ibn: <class 'numpy.ndarray'>, shape: (768, 576)
- Id: <class 'numpy.ndarray'>, shape: (768, 576)
- I_hsv: <class 'numpy.ndarray'>, shape: (768, 576, 3)
- Is: <class 'numpy.ndarray'>, shape: (768, 576)
- MSCN_window: <class 'numpy.ndarray'>, shape: (7, 7)
- mu: <cla

In [169]:
def test(input_variable, variable_name, mat_file='data/FADE_key_variables.mat'):
    """
    改进版测试函数：
    - 矩阵比较时显示具体数值
    - 标量比较时忽略(1,1)形状差异，简化输出
    """
    try:
        # 读取.mat文件
        mat_data = scipy.io.loadmat(mat_file)
        
        # 检查变量是否存在
        if variable_name not in mat_data:
            print(f"错误：变量 '{variable_name}' 不存在")
            return False
        
        mat_variable = mat_data[variable_name]
        
        # 转换为numpy数组
        input_array = np.array(input_variable)
        mat_array = np.array(mat_variable)
        
        # 判断是否为标量比较（输入是标量且mat变量是1x1数组）
        is_scalar_comparison = (input_array.shape == () and mat_array.shape == (1, 1)) or \
                              (input_array.shape == (1, 1) and mat_array.shape == ())
        
        if is_scalar_comparison:
            # 标量比较：简化处理
            print(f"比较标量值: 输入={input_variable}, Mat={(mat_array.item() if mat_array.size == 1 else mat_array)}")
            
            # 统一为标量进行比较
            input_scalar = input_array.item() if input_array.size == 1 else input_array
            mat_scalar = mat_array.item() if mat_array.size == 1 else mat_array
            
            are_equal = input_scalar == mat_scalar
        else:
            # 矩阵比较：显示详细信息
            print(f"输入变量形状: {input_array.shape}")
            print(f"Mat变量形状: {mat_array.shape}")
            # print(f"输入变量:\n{input_array}")
            # print(f"Mat变量:\n{mat_array}")
            
            # 检查形状是否相同
            if input_array.shape != mat_array.shape:
                print(f"形状不同：{input_array.shape} vs {mat_array.shape}")
                return False
            
            # 比较数值
            if np.issubdtype(input_array.dtype, np.floating) or np.issubdtype(mat_array.dtype, np.floating):
                are_equal = np.allclose(input_array, mat_array, rtol=1e-10, atol=1e-12)
            else:
                are_equal = np.array_equal(input_array, mat_array)
        
        # 输出结果
        if are_equal:
            print(f"✓ 变量 '{variable_name}' 数值相同")
            return True
        else:
            print(f"✗ 变量 '{variable_name}' 数值不同")
            if not is_scalar_comparison:
                # 只有矩阵比较时才显示详细差异
                if input_array.size <= 20:  # 小矩阵显示差异
                    diff = np.abs(input_array - mat_array)
                    print(f"差异矩阵:\n{diff}")
                else:
                    max_diff = np.max(np.abs(input_array - mat_array))
                    print(f"最大差异: {max_diff}")
            return False
            
    except Exception as e:
        print(f"出错: {e}")
        return False

In [ ]:
def test(input_variable, variable_name, mat_file='data/FADE_key_variables.mat', tolerance=1e-8):
    """
    改进版测试函数：
    - 矩阵比较时显示具体数值
    - 标量比较时忽略(1,1)形状差异，简化输出
    - 浮点数误差容忍度调整为1e-5
    
    参数:
    input_variable: 要比较的输入变量
    variable_name: MAT文件中的变量名
    mat_file: MAT文件路径
    tolerance: 浮点数比较的容忍度（默认1e-5）
    """
    try:
        # 读取.mat文件
        mat_data = scipy.io.loadmat(mat_file)
        
        # 检查变量是否存在
        if variable_name not in mat_data:
            print(f"错误：变量 '{variable_name}' 不存在")
            return False
        
        mat_variable = mat_data[variable_name]
        
        # 转换为numpy数组
        input_array = np.array(input_variable)
        mat_array = np.array(mat_variable)
        
        # 判断是否为标量比较（输入是标量且mat变量是1x1数组）
        is_scalar_comparison = (input_array.shape == () and mat_array.shape == (1, 1)) or \
                              (input_array.shape == (1, 1) and mat_array.shape == ())
        
        if is_scalar_comparison:
            # 标量比较：简化处理
            input_scalar = input_array.item() if input_array.size == 1 else input_array
            mat_scalar = mat_array.item() if mat_array.size == 1 else mat_array
            
            print(f"比较标量值: 输入={input_scalar}, Mat={mat_scalar}")
            
            # 浮点数比较使用容忍度
            if isinstance(input_scalar, (float, np.floating)) or isinstance(mat_scalar, (float, np.floating)):
                are_equal = np.isclose(input_scalar, mat_scalar, rtol=tolerance, atol=tolerance)
                if not are_equal:
                    diff = abs(input_scalar - mat_scalar)
                    print(f"差异: {diff:.2e} (容忍度: {tolerance:.1e})")
            else:
                are_equal = input_scalar == mat_scalar
        else:
            # 矩阵比较：显示详细信息
            print(f"输入变量形状: {input_array.shape}")
            print(f"Mat变量形状: {mat_array.shape}")
            
            # 检查形状是否相同
            if input_array.shape != mat_array.shape:
                print(f"形状不同：{input_array.shape} vs {mat_array.shape}")
                return False
            
            # 比较数值，使用调整后的容忍度
            if np.issubdtype(input_array.dtype, np.floating) or np.issubdtype(mat_array.dtype, np.floating):
                are_equal = np.allclose(input_array, mat_array, rtol=tolerance, atol=tolerance)
            else:
                are_equal = np.array_equal(input_array, mat_array)
            
            # 如果不相等，显示差异信息
            if not are_equal and not is_scalar_comparison:
                max_diff = np.max(np.abs(input_array - mat_array))
                mean_diff = np.mean(np.abs(input_array - mat_array))
                print(f"最大差异: {max_diff:.2e}, 平均差异: {mean_diff:.2e} (容忍度: {tolerance:.1e})")
                
                # 如果是小矩阵，显示详细差异
                if input_array.size <= 20:
                    diff = np.abs(input_array - mat_array)
                    print(f"差异矩阵:\n{diff}")
        
        # 输出结果
        if are_equal:
            print(f"✓ 变量 '{variable_name}' 数值相同 (容忍度: {tolerance:.1e})")
            return True
        else:
            print(f"✗ 变量 '{variable_name}' 数值不同")
            return False
            
    except Exception as e:
        print(f"出错: {e}")
        return False

In [171]:
# 导入一张图片

I = cv2.imread('data/test_image1.png')

image_path = 'data/test_image1.png'

I = Image.open(image_path)
I = I.convert('RGB')  # 确保是RGB格式
I = np.array(I)

# I = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)
# I = I.astype(np.float64)

test(I , 'I')

输入变量形状: (768, 576, 3)
Mat变量形状: (768, 576, 3)
✓ 变量 'I' 数值相同 (容忍度: 1.0e-02)


True

In [172]:
# Basic Setup

ps = 8
row, col, dim = I.shape
patch_row_num = row // ps
patch_col_num = col // ps
I = I[:patch_row_num * ps, :patch_col_num * ps, :3]

test(ps , 'ps')
test(row , 'row')
test(col , 'col')
test(dim , 'dim')
test(patch_row_num , 'patch_row_num')
test(patch_col_num , 'patch_col_num')
test(I , 'I')

比较标量值: 输入=8, Mat=8
✓ 变量 'ps' 数值相同 (容忍度: 1.0e-02)
比较标量值: 输入=768, Mat=768
✓ 变量 'row' 数值相同 (容忍度: 1.0e-02)
比较标量值: 输入=576, Mat=576
✓ 变量 'col' 数值相同 (容忍度: 1.0e-02)
比较标量值: 输入=3, Mat=3
✓ 变量 'dim' 数值相同 (容忍度: 1.0e-02)
比较标量值: 输入=96, Mat=96
✓ 变量 'patch_row_num' 数值相同 (容忍度: 1.0e-02)
比较标量值: 输入=72, Mat=72
✓ 变量 'patch_col_num' 数值相同 (容忍度: 1.0e-02)
输入变量形状: (768, 576, 3)
Mat变量形状: (768, 576, 3)
✓ 变量 'I' 数值相同 (容忍度: 1.0e-02)


True

In [173]:
# RGB and gray channel

R = I[:, :, 0].astype(np.float64)
G = I[:, :, 1].astype(np.float64)
B = I[:, :, 2].astype(np.float64)
Ig = 0.299 * R + 0.587 * G + 0.114 * B 


test(R , 'R')
test(G , 'G')
test(B , 'B')
test(Ig , 'Ig')


print(f"这是这里计算的 Ig: {Ig}")
print(f"这是 mat 文件中存储的 Ig: {mat_data['Ig']}")
print("可以发现这里的差异实际上是小数导致的。所以 Ig 的差异可以无视")

输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'R' 数值相同 (容忍度: 1.0e-02)
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'G' 数值相同 (容忍度: 1.0e-02)
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'B' 数值相同 (容忍度: 1.0e-02)
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
最大差异: 5.04e-01, 平均差异: 2.46e-01 (容忍度: 1.0e-02)
✗ 变量 'Ig' 数值不同
这是这里计算的 Ig: [[157.712 158.527 157.527 ... 165.303 164.118 164.716]
 [158.413 158.527 159.114 ... 165.303 166.118 165.542]
 [159.413 160.413 158.527 ... 166.118 164.232 164.357]
 ...
 [176.142 169.729 153.832 ...  65.92   64.675  61.017]
 [143.843 117.957  92.544 ...  63.137  65.963  65.305]
 [125.555  89.647  67.772 ...  63.425  57.707  55.821]]
这是 mat 文件中存储的 Ig: [[158 159 158 ... 165 164 165]
 [158 159 159 ... 165 166 166]
 [159 160 159 ... 166 164 164]
 ...
 [176 170 154 ...  66  65  61]
 [144 118  93 ...  63  66  65]
 [126  90  68 ...  63  58  56]]
可以发现这里的差异实际上是小数导致的。所以 Ig 的差异可以无视


In [174]:
# Dark channel prior image: Id, pixel-wise, scaled to [0 1]

Irn = R / 255.0  # 归一化红色通道
Ign = G / 255.0  # 归一化绿色通道  
Ibn = B / 255.0  # 归一化蓝色通道
Id = np.minimum(np.minimum(Irn, Ign), Ibn)

test(Id , "Id")

输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'Id' 数值相同 (容忍度: 1.0e-02)


True

In [175]:
# HSV color space: saturation image: Is

from matplotlib.colors import rgb_to_hsv

I_normalized = I.astype(np.float32) / 255.0
I_hsv = rgb_to_hsv(I_normalized)  # 转换为HSV
Is = I_hsv[:, :, 1]  # 提取饱和度通道（第2个通道，索引为1）

test(I_hsv , "I_hsv")
test(Is , "Is")

输入变量形状: (768, 576, 3)
Mat变量形状: (768, 576, 3)
✓ 变量 'I_hsv' 数值相同 (容忍度: 1.0e-02)
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'Is' 数值相同 (容忍度: 1.0e-02)


True

In [176]:
# MSCN

def create_gaussian_window(size=7, sigma=None):
    """
    创建高斯窗口，等效于MATLAB的fspecial('gaussian', 7, 7/6)
    
    参数:
    size: 窗口大小（默认7）
    sigma: 标准差（默认为size/6）
    
    返回:
    归一化的高斯窗口
    """
    if sigma is None:
        sigma = size / 6.0
    
    # 创建高斯核
    # 方法1：使用numpy手动创建（与MATLAB最接近）
    center = size // 2
    x, y = np.mgrid[0:size, 0:size]
    x = x - center
    y = y - center
    
    # 高斯函数
    gaussian_kernel = np.exp(-(x**2 + y**2) / (2 * sigma**2))
    
    # 归一化
    gaussian_kernel = gaussian_kernel / np.sum(gaussian_kernel)
    
    return gaussian_kernel


MSCN_window = create_gaussian_window(size=7, sigma=7/6)
test(MSCN_window , "MSCN_window")

# 计算局部均值（等效于imfilter(Ig, MSCN_window, 'replicate')）
mu = ndimage.convolve(Ig, MSCN_window, mode='nearest')

# 计算均值的平方
mu_sq = mu * mu

# 计算局部方差和标准差
# 等效于imfilter(Ig.*Ig, MSCN_window, 'replicate') - mu_sq
Ig_sq_filtered = ndimage.convolve(Ig * Ig, MSCN_window, mode='nearest')
sigma = np.sqrt(np.abs(Ig_sq_filtered - mu_sq))

# MSCN处理（Mean Subtracted Contrast Normalized）
MSCN = (Ig - mu) / (sigma + 1)

# 计算变异系数
cv = sigma / mu



test(MSCN_window , "MSCN_window")
test(mu , "mu")

print(f"这是这里计算的 mu: {mu}")
print(f"这是 mat 文件中存储的 mu: {mat_data['mu']}")

test(sigma , "sigma")
test(cv , "cv")

输入变量形状: (7, 7)
Mat变量形状: (7, 7)
✓ 变量 'MSCN_window' 数值相同 (容忍度: 1.0e-02)
输入变量形状: (7, 7)
Mat变量形状: (7, 7)
✓ 变量 'MSCN_window' 数值相同 (容忍度: 1.0e-02)
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'mu' 数值相同 (容忍度: 1.0e-02)
这是这里计算的 mu: [[158.19787702 158.29116846 158.29321074 ... 164.9974207  164.92706085
  164.86414605]
 [158.65532247 158.71027634 158.66390445 ... 165.26426256 165.12608553
  164.96453368]
 [159.08920858 159.10086443 158.98597427 ... 165.52333846 165.20477504
  164.8446575 ]
 ...
 [149.70244913 139.8424462  129.68712824 ...  66.8153632   64.7838107
   62.09313311]
 [136.77537729 124.30591825 113.60644185 ...  63.77131471  62.84358926
   61.40697193]
 [122.22041405 107.80862342  97.92085461 ...  61.63965148  60.7382769
   59.39657183]]
这是 mat 文件中存储的 mu: [[158.35963449 158.51515341 158.47373035 ... 164.84029882 164.88955976
  164.98774359]
 [158.63658088 158.80860909 158.79047103 ... 165.12449742 165.04865491
  164.99468439]
 [158.98913288 159.15253813 159.11718261 ... 165.39704195 165

True

In [177]:
# rg and by channel
rg = R-G;
by = 0.5*(R+G)-B;

test(rg , "rg")
test(by , "by")

输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'rg' 数值相同 (容忍度: 1.0e-02)
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'by' 数值相同 (容忍度: 1.0e-02)


True